In [ ]:
# Rolling Portfolio Optimisation Loop didn't work s


covariance_lookback = 60 #As done in the paper


portfolio_weights_over_time = []

test_period_dates = all_predictions_df.index

print("Starting daily portfolio optimization...")

for current_date in test_period_dates:

    # This vector is the 'expected returns' for the optimizer
    predicted_returns_today = all_predictions_df.loc[current_date]

   
    # We look at the historical data available up to the day *before* today
    try:
        end_loc = historical_returns.index.get_loc(current_date)
        start_loc = end_loc - covariance_lookback
        
        # Ensure we have enough historical data
        if start_loc < 0:
            continue
            
        rolling_historical_returns = historical_returns.iloc[start_loc:end_loc]
        S = risk_models.risk_matrix(rolling_historical_returns, method='ledoit_wolf')
    except (KeyError, IndexError):
        # This handles cases where the date might not align perfectly or we run out of data
        print(f"Could not find historical data for date {current_date}. Skipping.")
        continue

    # This uses your predictions (predicted_returns_today) and the calculated risk (S)
    ef = EfficientFrontier(predicted_returns_today, S)
    
    # Add constraints as specified in the paper
    ef.add_constraint(lambda w: np.sum(w) == 1) # Weights must sum to 1
    ef.add_constraint(lambda w: w >= 0)         # Long-only (no shorting)

    try:
        # Solve for the weights that maximize the Sharpe ratio
        weights = ef.max_sharpe()
        cleaned_weights = ef.clean_weights() # Tidy up small weights
        portfolio_weights_over_time.append(cleaned_weights)
    except (OptimizationError, ValueError) as e:
        # If optimization fails (e.g., all predicted returns are negative),
        # we'll use a fallback of an equal-weight portfolio for that day.
        
        print(f"Could not optimize for date {current_date}. Using fallback. Reason: {e}")
        num_assets = len(assets)
        fallback_weights = {asset: 1/num_assets for asset in assets}
        portfolio_weights_over_time.append(fallback_weights)

# Convert the list of weights into a clean DataFrame
weights_df = pd.DataFrame(
    portfolio_weights_over_time, 
    index=test_period_dates[:len(portfolio_weights_over_time)] # Ensure index alignment
)

print("\n--- Portfolio optimization complete. ---")
print("Daily optimal weights have been calculated.")
print(weights_df.head())
